In [ ]:
import gzip
import struct
import csv

In [ ]:
Current_time=0;
length=1;
HOUR = 3600000000000; # 1 hour to nano seconds
Last_time =HOUR*3;
END = False;
input_file='01302019.NASDAQ_ITCH50.gz';
allOrders={}#declared as an dictionary of list
csvallOrders={}#declared as an dictionary of list
Executed_orders={}#declared as an dictionary of list of list
def read_binary_file(size):
    read = bin_data.read(size);
    return read;

def system_event_message(msg):
    global END
    global Current_time
    msg_type = 'S';
    temp = struct.unpack('>HH6sc',msg);    
    new_msg = struct.pack('>sHH2s6sc',msg_type,temp[0],temp[1],'x\00x\00',temp[2],temp[3]);
    val = struct.unpack('>cHHQc',new_msg);
    val = list(val);
    if(val[4] == 'E'):
        END = True;
    if(Current_time/float(HOUR)>13):
        print(val,msg_type)    
    return;

def stock_directory_message(msg):
    msg_type = 'R';
    temp = struct.unpack('>4s6s10cI9cIc',msg);
    new_msg = struct.pack('>s4s2s6s10sI9sIs',msg_type,temp[0],'\x00\x00',temp[1],''.join(list(temp[2:12])),temp[12],''.join(list(temp[13:22])),temp[22],temp[23]);
    val = struct.unpack('>sHHQ8sssIss2ssssssIs',new_msg);
    val = list(val);
    
    return ;
	
def stock_trading_action(msg):
    msg_type = 'H';
    temp = struct.unpack('>4s6s14c',msg);
    new_msg = struct.pack('>s4s2s6s14s',msg_type,temp[0],'\x00\x00',temp[1],''.join(list(temp[2:16])));
    val = struct.unpack('>sHHQ8sss4s',new_msg);
    val = list(val);
    return ;
	
def short_sale_price_test(msg):
    msg_type = 'Y';
    temp = struct.unpack('>4s6s9c',msg);
    new_msg = struct.pack('>s4s2s6s9s',msg_type,temp[0],'\x00\x00',temp[1],''.join(list(temp[2:11])));
    val = struct.unpack('>sHHQ8ss',new_msg);
    val = list(val);
    return ;
	
def market_participation_position(msg):
    msg_type = 'L';
    temp = struct.unpack('>4s6s15c',msg);
    new_msg = struct.pack('>s4s2s6s15s',msg_type,temp[0],'\x00\x00',temp[1],''.join(list(temp[2:17])));
    val = struct.unpack('>sHHQ4s8ssss',new_msg);
    val = list(val);
    return ;
	
def mwcb_decline_level_message(msg):
    msg_type = 'V';
    temp = struct.unpack('>4s6s24s',msg);
    new_msg = struct.pack('>s4s2s6s24s',msg_type,temp[0],'\x00\x00',temp[1],temp[2]);
    val = struct.unpack('>sHHQQQQ',new_msg);
    val = list(val);
    val[4:7] = map(float,val[4:7]);
    val[4:7] = map(lambda x:x/(pow(10,8)),val[4:7]);
    return ;
	
def mwcb_status_message(msg):
    msg_type = 'W';
    temp = struct.unpack('>4s6sc',msg);
    new_msg = struct.pack('>s4s2s6ss',msg_type,temp[0],'\x00\x00',temp[1],temp[2]);
    val = struct.unpack('>sHHQs',new_msg);
    val = list(val);
    return ;
	
def ipo_quoting_period_update(msg):
    msg_type = 'K';
    temp = struct.unpack('>4s6s8cIcI',msg);
    new_msg = struct.pack('>s4s2s6s8sIsI',msg_type,temp[0],'\x00\x00',temp[1],''.join(list(temp[2:10])),temp[10],temp[11],temp[12]);
    val = struct.unpack('>sHHQ8sIsI',new_msg);
    val = list(val);
    val[7] = float(val[7]);
    val[7] = val[7]/10000;
    return ;
	
def add_order_no_mpid(msg):
    global Current_time
    global csvallOrders
    global allOrders
    msg_type = 'A';
    temp = struct.unpack('>4s6sQcI8cI',msg);
    new_msg = struct.pack('>s4s2s6sQsI8sI',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3],temp[4],''.join(list(temp[5:13])),temp[13]);
    val = struct.unpack('>sHHQQsI8sI',new_msg);
    val = list(val);
    #val[4]=order id. val[6]= quantity val[1]=location id. val[8]= price    val[7]= stock symbol
    val[8] = float(val[8]);
    val[8] = val[8]/10000;    
    allOrders[val[4]]=[val[4],val[6],val[1],val[8],val[7]];
    #print(val)
    if val[1] in csvallOrders: 
        pass ;
    else:
        csvallOrders[val[1]]=[val[7]];
    Current_time=val[3];    
    
    return ;

def add_order_with_mpid(msg):
    global Current_time
    global csvallOrders
    global allOrders
    msg_type = 'F';
    temp = struct.unpack('>4s6sQcI8cI4c',msg);
    new_msg = struct.pack('>s4s2s6sQsI8sI4s',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3],temp[4],''.join(list(temp[5:13])),temp[13],''.join(list(temp[14:18])));
    val = struct.unpack('>sHHQQsI8sI4s',new_msg);
    val = list(val);
    #val[4]=order id. val[6]= quantity val[1]=location id. val[8]= price     val[7]= stock symbol
    val[8] = float(val[8]);
    val[8] = val[8]/10000;
    allOrders[val[4]]=[val[4],val[6],val[1],val[8],val[7]];    
    if val[1] in csvallOrders:         
        pass ;
    else:        
        csvallOrders[val[1]]=[val[7]];
    Current_time=val[3];
    return ;

def order_executed_message(msg):
    global Current_time
    global allOrders
    global Executed_orders
    msg_type = 'E';
    temp = struct.unpack('>4s6sQIQ',msg);
    new_msg = struct.pack('>s4s2s6sQIQ',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3],temp[4]);
    val = struct.unpack('>sHHQQIQ',new_msg);
    val = list(val);
    #val[1]= location id, val[4]= order id, val[5]=quantity, allOrders[val[4]][3]]= price
    #order is location id - 0 ,quantity - 1, price -2;
    if val[4] not in  allOrders:
        Current_time=val[3];
        return ;
    allOrders[val[4]][1]-=val[5];
    #dictionary of list of lists
    if val[1] in Executed_orders: 
        Executed_orders[val[1]]+=[[val[1],val[5],allOrders[val[4]][3]]];
    else:
        Executed_orders[val[1]]=[[val[1],val[5],allOrders[val[4]][3]]];
    #print(Executed_orders[val[1]]);
    #delete if no more quantity left
    if allOrders[val[4]][1] ==0:
        del allOrders[val[4]];    
#     for i,value in enumerate(Executed_orders[val[1]]):
#         print(i,value[2]);
    Current_time=val[3];
    
    return ;
	
def order_executed_price_message(msg):
    global Current_time
    global allOrders
    global Executed_orders
    msg_type = 'C';
    temp = struct.unpack('>4s6sQIQcI',msg);
    new_msg = struct.pack('>s4s2s6sQIQsI',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3],temp[4],temp[5],temp[6]);
    val = struct.unpack('>sHHQQIQsI',new_msg);
    val = list(val);    
    #val[1]= location id, val[4]= order id, val[5]=quantity, val[8]= price(assumption)
    #order is location id - 0 ,quantity - 1, price -2;
    val[8] = float(val[8]);
    val[8] = val[8]/10000; 
    if val[4] not in  allOrders:
        Current_time=val[3];
        return ;
    allOrders[val[4]][1]-=val[5];
    #dictionary of list of lists
    if val[1] in Executed_orders: 
        Executed_orders[val[1]]+=[[val[1],val[5],val[8]]];
    else:
        Executed_orders[val[1]]=[[val[1],val[5],val[8]]];
    #delete if no more quantity left
    if allOrders[val[4]][1] ==0:
        del allOrders[val[4]];    
    Current_time=val[3];    
    
    return ;

def order_cancel_message(msg):
    global Current_time;
    global allOrders
    msg_type = 'X';
    temp = struct.unpack('>4s6sQI',msg);
    new_msg = struct.pack('>s4s2s6sQI',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3]);
    val = struct.unpack('>sHHQQI',new_msg);    
    val = list(val);    
    if val[4] in allOrders:
        allOrders[val[4]][1]-=val[5];
    else:
        Current_time=val[3];
        return ;
    if allOrders[val[4]][1]==0:
#         print("deleted")
        del allOrders[val[4]];
    Current_time=val[3];

    return ;

def order_delete_message(msg):
    global Current_time;
    global allOrders;
    msg_type = 'D';
    temp = struct.unpack('>4s6sQ',msg);
    new_msg = struct.pack('>s4s2s6sQ',msg_type,temp[0],'\x00\x00',temp[1],temp[2]);
    val = struct.unpack('>sHHQQ',new_msg);
    val = list(val);
    #print(val[4])
    if val[4] in allOrders:        
        del allOrders[val[4]];    
    Current_time=val[3];
    
    return ;
	
def order_replace_message(msg):
    global Current_time;
    global allOrders
    msg_type = 'U';
    temp = struct.unpack('>4s6sQQII',msg);
    new_msg = struct.pack('>s4s2s6sQQII',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3],temp[4],temp[5]);
    val = struct.unpack('>sHHQQQII',new_msg);
    val = list(val);
    val[7] = float(val[7]);
    val[7] = val[7]/10000;
    #print(val);
    #val[5] = order id, val[6] = quantity, val[1]=location id, val[7]= price
    #order id =0 , quantity =1 , location id =2, price =3;
    if val[4] in allOrders:
        allOrders[val[5]]=[val[5],val[6],val[1],val[7]];  
        del allOrders[val[4]];        
    else:
        allOrders[val[5]]=[val[5],val[6],val[1],val[7]];  
    #print(allOrders[val[5]]);
    Current_time=val[3];
       
    return ;
	
def trade_message(msg):
    global Current_time;
    msg_type = 'P';
    temp = struct.unpack('>4s6sQcI8cIQ',msg);
    new_msg = struct.pack('>s4s2s6sQsI8sIQ',msg_type,temp[0],'\x00\x00',temp[1],temp[2],temp[3],temp[4],''.join(list(temp[5:13])),temp[13],temp[14]);
    val = struct.unpack('>sHHQQsI8sIQ',new_msg);
    val = list(val);    
    val[8] = float(val[8]);
    val[8] = val[8]/10000;
    
    return ;
	
def cross_trade_message(msg):
    global Current_time;
    msg_type = 'Q';
    temp = struct.unpack('>4s6sQ8cIQc',msg);
    new_msg = struct.pack('>s4s2s6sQ8sIQs',msg_type,temp[0],'\x00\x00',temp[1],temp[2],''.join(list(temp[3:11])),temp[11],temp[12],temp[13]);
    val = struct.unpack('>sHHQQ8sIQs',new_msg);
    val = list(val);    
    val[6] = float(val[6]);
    val[6] = val[6]/10000;    
    
    return ;
	
def broken_trade_execution_message(msg):
    global Current_time;
    msg_type = 'B';
    temp = struct.unpack('>4s6sQ',msg);
    new_msg = struct.pack('>s4s2s6sQ',msg_type,temp[0],'\x00\x00',temp[1],temp[2]);
    val = struct.unpack('>sHHQQ',new_msg);
    val = list(val);    
    
    return ;
	
def net_order_imbalance_message(msg):
    global Current_time;
    msg_type = 'I';
    temp = struct.unpack('>4s6s16s9c12s2c',msg);
    new_msg = struct.pack('>s4s2s6s16s9s12s2s',msg_type,temp[0],'\x00\x00',temp[1],temp[2],''.join(list(temp[3:12])),temp[12],''.join(list(temp[13:15])));
    val = struct.unpack('>sHHQQQs8sIIIss',new_msg);
    val = list(val);
    
    val[8:11] = map(float,val[8:11]);
    val[8:11] = map(lambda x:x/10000,val[8:11]);
    return ;
	
def retail_price_improvement_indicator(msg):
    msg_type = 'N';    
    temp = struct.unpack('>4s6s9c',msg);
    new_msg = struct.pack('>s4s2s6s9s',msg_type,temp[0],'\x00\x00',temp[1],''.join(list(temp[2:11])));
    val = struct.unpack('>sHHQ8ss',new_msg);
    val = list(val);

    return ;

def print_csv():
    global length
    global csvallOrders    
    with open('VWAP.csv', mode='w') as VWAP_file:
        VWAP_writer = csv.writer(VWAP_file,dialect='excel', delimiter=',',lineterminator='\r', quotechar='"', quoting=csv.QUOTE_MINIMAL)        
        VWAP_writer.writerow(csvallOrders['.']);
        for row in csvallOrders:            
            while len(csvallOrders[row])<length:
                csvallOrders[row]+=[0];        
            if row == '.':
                pass;
            else:
                VWAP_writer.writerow(csvallOrders[row])        
                pass;
        if((length+4)%12==0):
            csvallOrders['.']+=[12];
        else:
            csvallOrders['.']+=[(length+4)%12];
    length+=1;        
    return;

def print_volume_weighted_average_price(time):
    global csvallOrders
    global Executed_orders
    print("Printing volume")  
    f= open("vwap.txt","a+")  
    f.write("time "+ str(time)+"\n\n")    
    for location_id in Executed_orders:
        volume = 0;
        price_wt = 0;
        for values in Executed_orders[location_id]:
            #print(values)
            volume+=values[1];
            price_wt+=values[1]*values[2];        
        print("Volume weighted average price of id "+str(location_id)+" = "+str(price_wt/float(volume)));
        out_value="Volume weighted average price of id "+str(location_id)+" = "+str(price_wt/float(volume))+"\n";
        csvallOrders[location_id]+=[price_wt/float(volume)];        
        f.write(out_value);
    print_csv();    
    f.close();
    return

In [ ]:
c=0;
csvallOrders['.']=['.'];
bin_data = gzip.open(input_file,'rb');
message_type = bin_data.read(1);
while message_type:
    global Current_time;
    if message_type == "S":
        message = read_binary_file(11);
        system_event_message(message);
    elif message_type == "R":
        message = read_binary_file(38);
        stock_directory_message(message);
    elif message_type == "H":#not done
        message = read_binary_file(24);
        stock_trading_action(message);""
    elif message_type == "Y":#not done
        message = read_binary_file(19);
        short_sale_price_test(message);
    elif message_type == "L": #NOT done
        message = read_binary_file(25);
        market_participation_position(message);
    elif message_type == "V":
        message = read_binary_file(34);
        mwcb_decline_level_message(message);
    elif message_type == "W":
        message = read_binary_file(11);
        mwcb_status_message(message);
    elif message_type == "K":
        message = read_binary_file(27);
        ipo_quoting_period_update(message);
    elif message_type == "A":
        message = read_binary_file(35);
        add_order_no_mpid(message);
    elif message_type == "F":
        message = read_binary_file(39);
        add_order_with_mpid(message);
    elif message_type == "J":
        message = read_binary_file(34); 
    elif message_type == "E":
        message = read_binary_file(30);
        order_executed_message(message);
    elif message_type == "C":
        message = read_binary_file(35);
        order_executed_price_message(message);
    elif message_type == "X":
        message = read_binary_file(22);
        order_cancel_message(message);
    elif message_type == "D":
        message = read_binary_file(18);
        order_delete_message(message);
    elif message_type == "U":
        message = read_binary_file(34);
        order_replace_message(message);
    elif message_type == "P":
        message = read_binary_file(43);
        trade_message(message);
    elif message_type == "Q":
        message = read_binary_file(39);
        cross_trade_message(message);
    elif message_type == "h":
        message = read_binary_file(20);        
    elif message_type == "B":
        message = read_binary_file(18);
        broken_trade_execution_message(message);
    elif message_type == "I":
        message = read_binary_file(49);
        net_order_imbalance_message(message);
    elif message_type == "N":
        message = read_binary_file(19);
        retail_price_improvement_indicator(message);
    
    if(Current_time - Last_time-HOUR>0):        
        print(Last_time,Current_time,"last and current time")        
        Last_time=Current_time;        
        print_volume_weighted_average_price(Last_time/float(HOUR))    
    if(END== True):
        break;
    c+=1;
    message_type = bin_data.read(1);
    if c%1000000==0:
        #print("c = "+str(c))
        print((Current_time-Last_time-HOUR)/float(HOUR),(length+4)%12);
bin_data.close();

In [7]:
Current_time/float(HOUR)

19.999999163325278

In [6]:
Last_time/float(HOUR)


19.000004530534444

In [ ]:
for row in csvallOrders:    
    print(csvallOrders[row],len(csvallOrders[row]),length)
print(len(csvallOrders))

In [ ]:
csvallOrders.values()

In [ ]:
13%12